In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
import numpy as np
import pandas as pd
import re
import shutil
import os
import unicodedata
import urllib3
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [61]:
http = urllib3.PoolManager()

url ='http://www.manythings.org/anki/kor-eng.zip'

filename = 'kor-eng.zip'

path = os.getcwd()

zipfilename = os.path.join(path, filename)

with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       

    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
  zip_ref.extractall(path)

In [62]:
lines = pd.read_csv('kor.txt', names=['en','ko'], sep='\t', index_col=False)
lines.shape

(3729, 2)

In [63]:
lines.head(10)

,en,ko
0,Go.,가.
1,Hi.,안녕.
2,Run!,뛰어!
3,Run.,뛰어.
4,Who?,누구?
5,Wow!,우와!
6,Fire!,쏴!
7,Help!,도와줘!
8,Jump!,점프!
9,Jump.,점프해.


In [64]:
lines.ko = lines.ko.apply(lambda x: '\t '+x+' \n')
lines[1000:1100]

,en,ko
1000,How may I serve you?,\t 어떻게 도와드릴까요? \n
1001,I acted like a fool.,\t 나는 바보같이 굴었어. \n
1002,I always study hard.,\t 나는 늘 열심히 공부해. \n
1003,I did nothing wrong.,\t 난 잘못한 거 없어. \n
1004,I don't know either.,\t 나도 몰라. \n
...,...,...
1095,I hope Tom is coming.,\t 톰이 오고 있길 바라. \n
1096,I major in economics.,\t 난 경제학을 전공하고 있어. \n
1097,I meant the opposite.,\t 내 의도는 그 반대였다. \n
1098,I met Tom after work.,\t 난 퇴근 후 톰을 만났다. \n


In [65]:
# en & ko 글자 집합 구축
en_vocab = set()
for line in lines.en :
  for char in line:
    en_vocab.add(char)

#print(en_vocab)

ko_vocab = set()
for line in lines.ko :
  for char in line:
    ko_vocab.add(char)

#print(ko_vocab)

{'a', ' ', 'n', '%', 'z', 'd', 'Q', '-', '3', 's', '$', '?', '9', 'I', '5', 'o', 'c', 'u', 'W', 'C', 'e', 'R', 'J', 'F', 'p', 'x', 'A', 'l', 'T', 'y', 'b', 'q', 'E', 'r', '!', 'M', 'g', ',', 'i', 'O', "'", 'B', 'w', '0', 'j', 'U', '"', 'P', 'h', 'ï', 't', '.', 'L', '°', ':', 'k', 'v', '2', 'V', ';', '8', '7', 'K', 'D', 'G', '1', '6', 'H', 'N', 'Y', 'S', 'f', 'm', '4'}
{'룹', '잃', '듣', '척', '낸', '즈', '젠', '헉', '흥', '옥', '렀', '짝', '같', '굶', '습', '명', '깡', '픔', '얗', '게', '닮', '는', '쏴', '필', '닫', '붙', '정', '즘', '혼', '펐', '모', '창', '덤', '회', '0', '엽', '렛', '"', '밟', '녀', '껍', '며', '수', '짜', '찔', '뢰', '읍', '픈', '않', 'H', '총', '빼', '받', '디', '논', '앗', '번', '극', '탐', '팀', '내', '계', '결', '돈', '으', '토', '켤', '언', '폰', '네', '액', '득', '릴', '른', '함', '십', '먼', '샌', '럼', '해', '획', '훔', '트', '턱', '웨', '넣', '세', '치', '핸', '환', '달', '있', '떠', '드', '굽', '했', 'i', '알', '익', '닥', '핑', '풀', 'h', '쳐', '옷', '즐', '꽃', '끊', '우', '램', '린', '땋', '춤', '졸', '뚱', '차', 'N', '벌', '돌', '섯', '칠', '쏘', '깊', '그', '행', '끄'

In [66]:
en_vocab_size = len(en_vocab)+1
ko_vocab_size = len(ko_vocab)+1
print(en_vocab_size)
print(ko_vocab_size)

75
915


In [69]:
en_vocab = sorted(list(en_vocab))
ko_vocab = sorted(list(ko_vocab))
print(en_vocab)
print(ko_vocab)

[' ', '!', '"', '$', '%', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '°', 'ï']
['\t', '\n', ' ', '!', '"', '%', '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'H', 'M', 'N', 'T', 'a', 'd', 'h', 'i', 'm', 'o', 'p', 'r', 't', 'y', '°', '가', '각', '간', '갇', '갈', '감', '갑', '값', '갔', '강', '갖', '같', '개', '객', '갰', '걀', '걔', '거', '걱', '건', '걷', '걸', '검', '겁', '것', '게', '겐', '겠', '겨', '격', '겪', '견', '결', '겼', '경', '계', '고', '곡', '곤', '곧', '골', '곰', '곱', '곳', '공', '과', '관', '광', '괜', '괴', '굉', '교', '구', '국', '군', '굳', '굴', '굶', '굼', '굽', '궁', '권', '귀', '귄', '규', '그', '극', '근', '글', '금', '급', '긋', '긍', '기', '긴', '길', '깊', '까', '깎', '깐', '깔', '깜', '

In [73]:
# char 별 idx 매칭
en_idx = dict([(word, i+1) for i, word in enumerate(en_vocab)])
ko_idx = dict([(word, i+1) for i, word in enumerate(ko_vocab)])

print(en_idx)
print(ko_idx)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, "'": 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '?': 22, 'A': 23, 'B': 24, 'C': 25, 'D': 26, 'E': 27, 'F': 28, 'G': 29, 'H': 30, 'I': 31, 'J': 32, 'K': 33, 'L': 34, 'M': 35, 'N': 36, 'O': 37, 'P': 38, 'Q': 39, 'R': 40, 'S': 41, 'T': 42, 'U': 43, 'V': 44, 'W': 45, 'Y': 46, 'a': 47, 'b': 48, 'c': 49, 'd': 50, 'e': 51, 'f': 52, 'g': 53, 'h': 54, 'i': 55, 'j': 56, 'k': 57, 'l': 58, 'm': 59, 'n': 60, 'o': 61, 'p': 62, 'q': 63, 'r': 64, 's': 65, 't': 66, 'u': 67, 'v': 68, 'w': 69, 'x': 70, 'y': 71, 'z': 72, '°': 73, 'ï': 74}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '%': 6, '(': 7, ')': 8, ',': 9, '-': 10, '.': 11, '/': 12, '0': 13, '1': 14, '2': 15, '3': 16, '4': 17, '5': 18, '6': 19, '7': 20, '8': 21, '9': 22, ':': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'H': 29, 'M': 30, 'N': 31, 'T': 32, 'a': 33, 'd': 34, 'h': 35, 'i': 36, 'm': 37, 'o': 38, 'p': 3

In [74]:
# 입력 구성
encoder_input = []
for line in lines.en :
  temp_X = []
  for w in line:
    temp_X.append(en_idx[w])   #char - int 변환
  encoder_input.append(temp_X)
print(encoder_input[:10])

[[29, 61, 9], [30, 55, 9], [40, 67, 60, 2], [40, 67, 60, 9], [45, 54, 61, 22], [45, 61, 69, 2], [28, 55, 64, 51, 2], [30, 51, 58, 62, 2], [32, 67, 59, 62, 2], [32, 67, 59, 62, 9]]


In [75]:
# 출력 구성
decoder_input = []
for line in lines.ko :
  temp_X = []
  for w in line:
    temp_X.append(ko_idx[w])   #char - int 변환
  decoder_input.append(temp_X)
print(decoder_input[:10])

[[1, 3, 44, 11, 3, 2], [1, 3, 551, 195, 11, 3, 2], [1, 3, 289, 573, 4, 3, 2], [1, 3, 289, 573, 11, 3, 2], [1, 3, 206, 96, 24, 3, 2], [1, 3, 615, 606, 4, 3, 2], [1, 3, 539, 4, 3, 2], [1, 3, 243, 606, 690, 4, 3, 2], [1, 3, 666, 848, 4, 3, 2], [1, 3, 666, 848, 862, 11, 3, 2]]


In [78]:
# ko column 의 \t 제거
decoder_target = []
for line in lines.ko :
  t=0
  temp_X = []
  for w in line:
    if t>0:
      temp_X.append(ko_idx[w])
    t=t+1
  decoder_target.append(temp_X)
print(decoder_target[:10])

[[3, 44, 11, 3, 2], [3, 551, 195, 11, 3, 2], [3, 289, 573, 4, 3, 2], [3, 289, 573, 11, 3, 2], [3, 206, 96, 24, 3, 2], [3, 615, 606, 4, 3, 2], [3, 539, 4, 3, 2], [3, 243, 606, 690, 4, 3, 2], [3, 666, 848, 4, 3, 2], [3, 666, 848, 862, 11, 3, 2]]


In [79]:
# max length
max_en_len = max([len(line) for line in lines.en])
max_ko_len = max([len(line) for line in lines.ko])

print(max_en_len)
print(max_ko_len)

537
300


In [81]:
# 최대 길이를 10으로 나누어 사용 (학습 시간 단축)\n",
max_en_len //= 10
max_ko_len //= 10
print(max_en_len) 
print(max_ko_len)  
encoder_input = pad_sequences(encoder_input, maxlen=max_en_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_ko_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_ko_len, padding='post')

53
30


In [82]:
# 원핫 벡터
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

# **seq2seq 모델** **구현**

In [84]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

In [92]:
encoder_inputs = Input(shape=(None, en_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c] #은닉 상태, 셀 상태

In [93]:
decoder_inputs = Input(shape=(None, ko_vocab_size)) 
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(ko_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

#adams
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.summary()

# embedding 계층 관련 학습!

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, None, 75)]   0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           [(None, None, 915)]  0                                            
__________________________________________________________________________________________________
lstm_9 (LSTM)                   [(None, 256), (None, 339968      input_12[0][0]                   
__________________________________________________________________________________________________
lstm_10 (LSTM)                  [(None, None, 256),  1200128     input_13[0][0]                   
                                                                 lstm_9[0][1]               

In [94]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=50, validation_split=0.2)

Epoch 1/50
47/47 [==============================] - 30s 575ms/step - loss: 2.8160 - val_loss: 4.6562
Epoch 2/50
47/47 [==============================] - 26s 555ms/step - loss: 2.3478 - val_loss: 4.2339
Epoch 3/50
47/47 [==============================] - 26s 555ms/step - loss: 2.2260 - val_loss: 3.9024
Epoch 4/50
47/47 [==============================] - 26s 556ms/step - loss: 2.0389 - val_loss: 3.7397
Epoch 5/50
47/47 [==============================] - 26s 553ms/step - loss: 1.9271 - val_loss: 3.3868
Epoch 6/50
47/47 [==============================] - 26s 556ms/step - loss: 1.7786 - val_loss: 3.4240
Epoch 7/50
47/47 [==============================] - 26s 560ms/step - loss: 1.6697 - val_loss: 3.3826
Epoch 8/50
47/47 [==============================] - 26s 560ms/step - loss: 1.6026 - val_loss: 3.3363
Epoch 9/50
47/47 [==============================] - 26s 558ms/step - loss: 1.5542 - val_loss: 3.3825
Epoch 10/50
47/47 [==============================] - 26s 562ms/step - loss: 1.5027 - val_lo

In [95]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)
encoder_model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, None, 75)]        0         
_________________________________________________________________
lstm_9 (LSTM)                [(None, 256), (None, 256) 339968    
Total params: 339,968
Trainable params: 339,968
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#hist = model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=50, validation_split=0.2)
#train_acc=hist.history['acc']

In [96]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, None, 915)]  0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           [(None, 256)]        0                                            
__________________________________________________________________________________________________
lstm_10 (LSTM)                  [(None, None, 256),  1200128     input_13[0][0]                   
                                                                 input_14[0][0]             

In [98]:
idx_to_en = dict((i, char) for char, i in en_idx.items())
idx_to_ko = dict((i, char) for char, i in ko_idx.items())

In [102]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, ko_vocab_size))
    target_seq[0, 0, ko_idx['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx_to_ko[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_ko_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, ko_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [106]:
import random
for i in range(10): # 입력 문장의 인덱스
    seq_index = random.randint(10,300)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.en[seq_index])
    print('정답 문장:', lines.ko[seq_index][1:len(lines.ko[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Tom forgot.
정답 문장:  톰이 잊었어. 
번역기가 번역한 문장:  톰이 웃었어. 
-----------------------------------
입력 문장: Say hello.
정답 문장:  인사해. 
번역기가 번역한 문장:  그만 사람이 있어. 
-----------------------------------
입력 문장: Tom lost.
정답 문장:  톰이 졌어. 
번역기가 번역한 문장:  톰이 이겼어. 
-----------------------------------
입력 문장: Kill them.
정답 문장:  그들을 죽여라. 
번역기가 번역한 문장:  그만 사람이 있어. 
-----------------------------------
입력 문장: Hurry back.
정답 문장:  빨리 와. 
번역기가 번역한 문장:  그만 사람이 있어! 
-----------------------------------
입력 문장: Tom fought.
정답 문장:  톰이 싸웠어. 
번역기가 번역한 문장:  톰이 웃었어. 
-----------------------------------
입력 문장: Tom helped.
정답 문장:  톰이 도와줬어. 
번역기가 번역한 문장:  톰이 웃었어. 
-----------------------------------
입력 문장: Bring wine.
정답 문장:  와인을 가져와. 
번역기가 번역한 문장:  이렇게 귀엽다니! 
-----------------------------------
입력 문장: Boys do cry.
정답 문장:  남자애도 운다. 
번역기가 번역한 문장:  그 사람은 사람이 웃었어. 
-----------------------------------
입력 문장: I'm ugly.
정답 문장:  나는 못 생겼다. 
번역기가 번역한 문장:  이렇게나 흥미롭다니! 
